### Треугольная платформа на колесах

In [1]:
%load_ext autoreload
%autoreload 2

import sympy as sp
from sympy import Derivative, Symbol, IndexedBase, Idx, Sum, Function, Matrix, Add, Eq
from sympy import diff, symbols, solve, linsolve, simplify, poly, pprint, factor, integrate, lambdify
from sympy import cos, sin, pi

# syspath trick
import sys
sys.path.append('../')

from tatarinov.database.singleton import db
db.set_root('../data/triangular_platform')


from tatarinov.utils.jupyter import display_list, display_obj
# from tatarinov.core.variables import *
from tatarinov.core.tatarinov import TatarinovSystem

from tatarinov.problems.triangular_platform.variables import *
from tatarinov.problems.triangular_platform.structure import *
from tatarinov.problems.triangular_platform.functions import euler, scalar

#### Псевдоскорости $\nu_1$, $\nu_2$
#### Координаты $\theta_1$, $\psi_1$, $\alpha$
#### Связи $\psi_1(\nu_1,\nu_2), \theta_1(\nu_1,\nu_2)$

In [2]:
# скорости точек

v['S'] = euler(S, P)
v['P'] = euler(P, C)
v['C'] = euler(C, D)
v['D'] = lambda i: Matrix([0,0,0])

In [3]:
# угловаые скорости

omega['platform']= lambda i: Derivative(alpha,t)*e['z']
omega['fork']    = lambda i: omega['platform'](i) + Derivative(theta[i],t)*e['z']
omega['wheel']   = lambda i: omega['fork'](i) + Derivative(psi[i],t)*n_wheel(i)

In [31]:
# псевдоскорости

eq['nu1'] = lambda i: scalar(v['S'](i), e['xi'])
eq['nu2'] = lambda i: scalar(v['S'](i), e['eta'])

In [32]:
# db.save(simplify(Matrix([Eq(nu[1], eq['nu1'](0)),
#                          Eq(nu[2], eq['nu2'](0))])),
#         'nu1,nu2',
#         description='Псевдоскорости nu1, nu2')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


In [14]:
# связи

eq['f(nu1,nu2)'] = lambda i: solve(
                      [Eq(eq['nu1'](i), nu[1]), Eq(eq['nu2'](i), nu[2])],
                      [Derivative(psi[i],t), Derivative(theta[i],t)],
                      dict=True)[0]; # возвращает словарь с выражениями для diff(psi) и diff(theta)

eq['diff(psi)_nu']   = lambda i: eq['f(nu1,nu2)'](i)[Derivative(psi[i],t)]
eq['diff(theta)_nu'] = lambda i: eq['f(nu1,nu2)'](i)[Derivative(theta[i],t)]

In [15]:
# db.save(Matrix([Eq(Derivative(psi[0], t), eq['diff(psi)_nu'](0)),
#                 Eq(Derivative(theta[0], t), eq['diff(theta)_nu'](0))]),
#         'constraints',
#         description='Связи diff(psi), diff(theta) для колеса 0')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


### Лагранжиан для платформы


$$ L = T - V $$ следовательно
$$ T = T_{pl} + T_{fo} + T_{wh} $$ где
$$ T_{pl} = \frac{m_{pl} v_S^2}{2} + \frac{(J_{pl} \omega_{pl}, \omega_{pl})}{2} $$
$$ T_{fo} = 0$$
$$ T_{wh} = \frac{m_{wh} v_S^2}{2} + \frac{(J_{wh} \omega_{wh}, \omega_{wh})}{2} $$

### Платформа

In [16]:
# Момент инерции платформы

J['platform'] = eye(3, 3) * a

In [26]:
# db.save(J['platform'],
#         'J_platform',
#         description='Момент инерации платформы относительно связанной системы отсчета')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


### Колесо

In [17]:
# Момент инерции колесо

J['wheel'] = Matrix([[b,0,0],[0,c,0],[0,0,b]])

In [23]:
# db.save(J['wheel'],
#         'J_wheel',
#         description='Момент инерации колеса относительно связанной системы отсчета')

PWD: /home/alice/Documents/Course/.bak/16apr2020/tatarinov_equation/notebooks, Database root: ../data/triangular_platform


_В одинаковых базисах нужно записывать угловую скорость и момент инерции!_

In [19]:
# Угловая скорость колеса в СО связанной с колесом

omega['wheel_in'] = lambda i: psi[i].diff(t)*e['y'] + (theta[i].diff(t) + alpha.diff(t))*e['z']

In [22]:
# db.save(omega['wheel_in'](0),
#         'wheel_in0',
#         description='Угловая скорость колеса в связанной системе отсчета')